# placeholder, embedding_lookup 和dropout

```
本篇博客介绍了placeholder, embedding_lookup 和dropout
```

In [1]:
import tensorflow as tf
import numpy as np
sess = tf.InteractiveSession()

## 1.placeholder

placeholder在笔记一中的feed机制中提到，是graph的数据输入。

原理很简单，下面的实验关注传入的tensor的shape问题：

如果传入的数据的shape是[3,3]，则placeholder的shape可以是[3,3]或者[3,None]；只是[3]会报错。

In [2]:
indata = tf.placeholder(tf.int32, shape=[3,None])

In [3]:
mat1 = tf.reshape(tf.range(1, 10, name="m1"), shape=[3, 3])

In [4]:
out = indata + mat1

In [5]:
out.eval(feed_dict={indata:np.arange(1,10).reshape(3,3)})

array([[ 2,  4,  6],
       [ 8, 10, 12],
       [14, 16, 18]], dtype=int32)

## 2.embedding_lookup

其实embedding_lookup的原理很简单，相当于在np.array中直接采用下标数组获取数据。

细节是返回的tensor的dtype和传入的被查询的tensor的dtype保持一致；和ids的dtype无关。

In [6]:
ids = [[1,2], [0,1]]
res = tf.nn.embedding_lookup(mat1, ids)
res.eval()

array([[[4, 5, 6],
        [7, 8, 9]],

       [[1, 2, 3],
        [4, 5, 6]]], dtype=int32)

In [7]:
mat2 = tf.cast(mat1, tf.float32)
mat2.eval()

array([[ 1.,  2.,  3.],
       [ 4.,  5.,  6.],
       [ 7.,  8.,  9.]], dtype=float32)

In [8]:
res2 = tf.nn.embedding_lookup(mat2, ids)
print res2.dtype
print res2.get_shape()
print tf.shape(res2)
print tf.shape(res2).eval()
res2.eval()

<dtype: 'float32'>
(2, 2, 3)
Tensor("Shape:0", shape=(3,), dtype=int32)
[2 2 3]


array([[[ 4.,  5.,  6.],
        [ 7.,  8.,  9.]],

       [[ 1.,  2.,  3.],
        [ 4.,  5.,  6.]]], dtype=float32)

In [9]:
mat3 = tf.placeholder(tf.float32, shape=[3, 3])
res3 = tf.nn.embedding_lookup(mat3, ids)
print res3.dtype

<dtype: 'float32'>


In [10]:
a1 = np.arange(1,10, dtype=np.int32).reshape(3,3)
a1

array([[1, 2, 3],
       [4, 5, 6],
       [7, 8, 9]], dtype=int32)

In [11]:
res3.eval(feed_dict={mat3:a1})

array([[[ 4.,  5.,  6.],
        [ 7.,  8.,  9.]],

       [[ 1.,  2.,  3.],
        [ 4.,  5.,  6.]]], dtype=float32)

## 3.dropout

dropout是常用的防止过拟合的技巧之一。

在tensorflow中，dropout函数接受一个x和一个keep_prob：x是输入的tensor，输出会随机按照keep_prob的概率让输出为0，其他不受影响的输出为输入*1/keep_prob。

这样可以保证输出的总量和输入保持一致；同时作为新的神经元的输入，可以忽略掉其中的一些神经元。

注意，x的dtype要和kepp_prob保持一致，一般都是float。

In [12]:
i=tf.range(0,10, dtype=tf.float32)
keep_prob = tf.placeholder("float")
i = tf.nn.dropout(x=i, keep_prob=keep_prob)

In [13]:
i.eval(feed_dict={keep_prob:0.7})

array([  0.        ,   1.42857146,   0.        ,   4.28571415,
         5.71428585,   7.14285707,   8.5714283 ,  10.        ,
        11.4285717 ,  12.8571434 ], dtype=float32)